In [14]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

# Load the weather data
weather_path = '/content/drive/My Drive/Diploma/Jan 2025 Term/BDM Project/Oct-Dec 24 Weather.xlsx'
weather_df = pd.read_excel(weather_path)

weather_df.rename(columns={'datetime': 'Date', 'precip': 'Precipitation'}, inplace=True)
weather_df['Date'] = pd.to_datetime(weather_df['Date'])
weather_df['Month'] = weather_df['Date'].dt.month
weather_df['Year'] = weather_df['Date'].dt.year
filtered = weather_df[(weather_df['Year'] == 2024) & (weather_df['Month'].isin([10, 11, 12]))]
filtered['Rainy_Day'] = filtered['Precipitation'] > 0
rainy_days = filtered.groupby('Month')['Rainy_Day'].sum().reset_index(name='Rainy_Days')

utility_costs = pd.DataFrame({
    'Month': [10, 11, 12],
    'Relevant_Cost': [41924, 26003, 31268]  # Oct: 23524+18400, Nov: 14238+11765, Dec: 17518+13750
})

regression_df = pd.merge(rainy_days, utility_costs, on='Month')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
print(utility_costs)

   Month  Relevant_Cost
0     10          41924
1     11          26003
2     12          31268


In [16]:
print(regression_df)

   Month  Rainy_Days  Relevant_Cost
0     10          16          41924
1     11           2          26003
2     12          10          31268


In [17]:
import statsmodels.api as sm

X = sm.add_constant(regression_df['Rainy_Days'])  # Independent: Rainy days
y = regression_df['Relevant_Cost']                # Dependent: Utility cost

model = sm.OLS(y, X).fit()
model.summary()

/usr/local/lib/python3.11/dist-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 3 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Relevant_Cost   R-squared:                       0.926
Model:                            OLS   Adj. R-squared:                  0.852
Method:                 Least Squares   F-statistic:                     12.53
Date:                Mon, 14 Apr 2025   Prob (F-statistic):              0.175
Time:                        12:47:34   Log-Likelihood:                -26.745
No. Observations:                   3   AIC:                             57.49
Df Residuals:                       1   BIC:                             55.69
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.269e+04   3439.490      6.598      0.096    -2.1e+04    6.64e+04
Rainy_Days  1111.3176    313.981      3.539      0.175   -2878.190    5100.825
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   1.365
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.516
Skew:                          -0.686   Prob(JB):                        0.772
Kurtosis:                       1.500   Cond. No.                         21.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""